In [7]:
import gym
env = gym.make('FrozenLake-v0')
print(env.__dict__)

{'env': <gym.envs.toy_text.frozen_lake.FrozenLakeEnv object at 0x7f34f0731b70>, 'action_space': Discrete(4), 'observation_space': Discrete(16), 'reward_range': (0, 1), 'metadata': {'render.modes': ['human', 'ansi']}, '_max_episode_steps': 100, '_elapsed_steps': None}


In [8]:
print(env.observation_space)
print(env.action_space)
print(env.nS)
print(env.nA)

Discrete(16)
Discrete(4)
16
4


env.P[x][y] returns the probability of each possible rewad and the next state, if the agent is in state 1 of frozenlake-env and decides to go to left

returns a list of transition tuples (prob,next_state,reward,done)

In [9]:
print(env.P[14][1])

[(0.3333333333333333, 13, 0.0, False), (0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True)]


In [10]:
print(env.P[14])

{0: [(0.3333333333333333, 10, 0.0, False), (0.3333333333333333, 13, 0.0, False), (0.3333333333333333, 14, 0.0, False)], 1: [(0.3333333333333333, 13, 0.0, False), (0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True)], 2: [(0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False)], 3: [(0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False), (0.3333333333333333, 13, 0.0, False)]}


## Iterative Policy Evaluation ## 

In [11]:
import numpy as np
def policy_eval(env,policy,gamma,theta=1e-5):
    values = np.zeros(env.nS)
    while True:
        delta = 0
        for s in range(env.nS):
            v = values[s]
            value_new = 0
            for a,prob_a in enumerate(policy[s]):
                for transition_prob,next_state,reward,_ in env.P[s][a]:
                    value_new += prob_a*(transition_prob*(reward+gamma*values[next_state]))
            values[s] = value_new
            delta = np.maximum(delta,np.abs(v-value_new))
        if delta < theta:
            return values


In [12]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
print(policy_eval(env,random_policy,1))


[0.013911   0.01161424 0.02094062 0.01046758 0.01623478 0.
 0.04074774 0.         0.03479961 0.08816698 0.14205099 0.
 0.         0.17581855 0.4392897  0.        ]


In [13]:
def q_from_v(env,V,state,gamma=1):
    q = np.zeros(env.nA)
    for action in env.P[state].keys():
        for prob,next_state,reward,_ in env.P[s][a_intended]:
            q[action] += prob * (reward + gamma * V[next_state])
    return q

In [14]:
def policy_improvement(env,V,gamma=1):
    policy = np.zeros([env.nS,env.nA])
    
    for state in range(env.nS):
        q_state = q_from_v(env,V,state,gamma)
        
        policy[s][np.argmax(q_state)] = 1
    return policy 

In [15]:
def policy_iteration(env,gamma=1,theta=1e-8):
    
    policy = np.ones([env.nS,env.nA]) / env.nA
    while True:
        V = policy_evaluation(env,policy,gamma,theta)
        
        next_policy = policy_improvement(env, V, gamma)
        
        delta = policy - next_policy
        
        policy  = next_policy.copy()
        
        if delta.all() < theta:
            break
    return policy,V

In [16]:
# obtain the optimal policy and optimal state-value function
policy_pi, V_pi = policy_iteration(env)

# print the optimal policy
print("\nOptimal Policy (LEFT = 0, DOWN = 1, RIGHT = 2, UP = 3):")
print(policy_pi,"\n")

plot_values(V_pi)

NameError: name 'policy_evaluation' is not defined